# Denoising of SAR data using Deep Learning
### Lorenz Bacca 
### Interdisciplinary Project in Data Science

This notebook is used to train and further evaluate the deep learning model that tries to remove speckle from SAR data.

### 1) Loading needed libraries and functions

In [17]:
from skimage import io
import os
import pandas as pd
import numpy as np

from model import Autoencoder
from utils import SARImageDataset
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn as nn

import math
from scipy.ndimage import uniform_filter
from statistics import variance

In [18]:
def mse(t, p, scale = 255):
    return np.mean(np.square(t * scale - p * scale))
    
def psnr(t, p, scale = 255):
    return 20 * math.log10(scale) - 10 * math.log10(mse(t, p))

def lee_filter(input_array, size=5):
    """
     Python Implementation of lee filter function. Removes speckle like noise from an input image using a weighted uniform filter

     Parameters
     __________
     input_array: str
         numpy array of the input image
     size: int
         size of the filter kernel

     Returns
     _______
     out_array: array
         filtered output array
     """
    # Calculate array mean, square mean and variance
    img_mean = uniform_filter(input_array, (size, size))
    img_sqr_mean = uniform_filter(input_array**2, (size, size))
    img_variance = img_sqr_mean - img_mean**2

    # Calculate the overall variance to determine the weights for the smoothing
    overall_variance = np.var(input_array.flatten())

    # remove speckle by smoothing the input. The higher the deviation from the mean, the higher the smoothing weight
    img_weights = img_variance / (img_variance + overall_variance)
    out_array = img_mean + img_weights * (input_array - img_mean)
    return out_array


In [19]:
def train(net, trainloader, testloader, NUM_EPOCHS):
    train_loss = []
    test_loss = []
    epochs = []
    for epoch in range(NUM_EPOCHS):
        epoch_train_loss = []
        running_loss = 0.0
        for data in trainloader:
            input_img, output_img = data
            input_img = input_img.to('cuda:0')
            output_img = output_img.to('cuda:0')
            optimizer.zero_grad()
            outputs = net(input_img)
            loss = criterion(outputs, output_img)

            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            prediction = outputs.cpu().detach().numpy()[0][0]
            target, x = data
            
            epoch_train_loss.append(psnr(target.numpy()[0][0], prediction))
        
        loss = running_loss / len(trainloader)
        train_loss.append(np.mean(epoch_train_loss))
        eval_loss = test(net, testloader)
        test_loss.append(eval_loss)
        epochs.append(epoch + 1)
        print('Epoch {} of {}, Train Loss: {:.10f}, Test Loss: {:.3f}'.format(
            epoch+1, NUM_EPOCHS, np.mean(epoch_train_loss), eval_loss))
        
    return (epochs, train_loss, test_loss)

In [20]:
def test(net, testloader):
    data = SARImageDataset(transform=transform, target_transform=transform, train = False)
    dataloader = DataLoader(data, batch_size=1, shuffle=True)
    
    psnr_loss = []
    
    for data in dataloader:
        train_features, train_labels = next(iter(dataloader))
        prediction = net(train_features.to('cuda:0'))
        
        source = train_features.numpy()[0][0]
        target = train_labels.numpy()[0][0]
        pred = prediction.cpu().detach().numpy()[0][0]
        
        psnr_loss.append(psnr(target, pred))
    
    return np.mean(psnr_loss)

In [21]:
net = Autoencoder()

transform = transforms.Compose([
    transforms.ToTensor(),
])

NUM_EPOCHS = 50
LEARNING_RATE = 5e-4
BATCH_SIZE = 1

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

### 2) Creating train and test dataset and executing training

In [ ]:
train_data = SARImageDataset(transform = transform, target_transform=transform, train = True)
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

test_data = SARImageDataset(transform = transform, target_transform=transform, train = False)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

net.to('cuda:0')
train_loss = train(net, train_dataloader, test_dataloader, NUM_EPOCHS)

### 3) Visual and quantitative evaluation

In [ ]:
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.cm as cm

data = SARImageDataset(transform=transform, target_transform=transform, train = False)
dataloader = DataLoader(data, batch_size=1, shuffle=True)

# Display image and label.
train_features, train_labels = next(iter(dataloader))
prediction = net(train_features.to('cuda:0'))

plt.figure()

#subplot(r,c) provide the no. of rows and columns
f, axarr = plt.subplots(1,6, figsize=(22,4), dpi=100) 

source = train_features.numpy()[0][0]
target = train_labels.numpy()[0][0]
pred = prediction.cpu().detach().numpy()[0][0]
pred_lee = lee_filter(source)
pred_target = target - pred
pred_lee_target = target - pred_lee

#labels
axarr[0].set_title('Input image')
axarr[1].set_title('Target image')
axarr[2].set_title('Lee filter prediction')
axarr[3].set_title('DL model prediction')
axarr[4].set_title('Lee filter - target')
axarr[5].set_title('DL model - target')

# use the created array to output your multiple images
axarr[0].imshow(source, cmap=cm.gray)
axarr[1].imshow(target, cmap=cm.gray)
axarr[2].imshow(pred_lee, cmap=cm.gray)
axarr[3].imshow(pred, cmap=cm.gray)
axarr[4].imshow(pred_lee, cmap=cm.jet)
axarr[5].imshow(pred_target, cmap=cm.jet)

In [ ]:
mse_loss_model = []
psnr_loss_model = []

mse_loss_lee = []
psnr_loss_lee = []

for data in dataloader:
    train_features, train_labels = next(iter(dataloader))
    prediction = net(train_features.to('cuda:0'))
    
    source = train_features.numpy()[0][0]
    target = train_labels.numpy()[0][0]
    pred = prediction.cpu().detach().numpy()[0][0]
    
    pred_lee = lee_filter(source)
    
    psnr_loss_model.append(psnr(target, pred))
    mse_loss_model.append(mse(target, pred))
    
    psnr_loss_lee.append(psnr(target, pred_lee))
    mse_loss_lee.append(mse(target, pred_lee))

In [ ]:
print(np.median(psnr_loss_model))
print(np.median(psnr_loss_lee))

print(np.median(mse_loss_model))
print(np.median(mse_loss_lee))

### 4) Saving and loading model

In [ ]:
import torch
torch.save(net.state_dict(), 'model.pt')

In [ ]:
model = Autoencoder()
model.load_state_dict(torch.load('model.pt'))
model.cuda()